In [ ]:
# faceMask and faceMask2 hDDM face ratings analysis script: mask analyses of false negatives (failing to ID correct emotion)
# 22/03/21

# note: to be run within python 3.5 environment named "hddm" (from console: 'source activate hddm')
# confirm jupyter notebook is launching from "hddm" environment (see top-right: 'conda env:hddm')

# set up
import numpy as np
print(np.__version__) # should be 1.11.3

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from patsy import dmatrix

import hddm
print(hddm.__version__) # should be 0.6.0

In [ ]:
# load hDDM-processed data from CSV file into a NumPy structured array
data1 = hddm.load_csv('faceMask_hddm_proc.csv')
                     
# flip 'no' response RTs to be negative
data1 = hddm.utils.flip_errors(data1)

# check dataset
data1.head(10)

In [ ]:
# plot RTs for each participant
fig = plt.figure()
ax = fig.add_subplot(111, xlabel='RT', ylabel='count', title='RT distributions')
for i, subj_data in data1.groupby('subj_idx'):
    subj_data.rt.hist(bins=20, histtype='step', ax=ax)

In [ ]:
# subset dataset by emotion rating type (angry) and facial expression (angry)
aa1_data = data1.loc[(data1['emotionRating'] == 'angry') & (data1['expression'] == 'angry')]
aa1_data.head(10)

In [ ]:
# drop any subjects missing columns in design matrix (otherwise produces error)
aa1_data = aa1_data[(aa1_data.subj_idx != 'f89wrv54kh9lqrw')]

In [ ]:
# define model
m_aa1 = hddm.HDDMRegressor(aa1_data, "v ~ C(mask)", bias=True, p_outlier=0.05)

In [ ]:
# run model
m_aa1.sample(5000, burn=200, dbname='traces.db', db='pickle')
m_aa1.print_stats()
m_aa1.plot_posteriors()

In [ ]:
# save model stats
aa1_stats = m_aa1.gen_stats()
print(aa1_stats)
aa1_stats.to_csv('aa1_5000/faceMask_hddm_drift_maskFalseNegatives_aa1_5000.csv', index=True)

# save model posterior plots
m_aa1.plot_posteriors(path='aa1_5000/_posteriors', save=True)

In [ ]:
# plot model posteriors by mask
aa1_v_none, aa1_v_lower, aa1_v_upper = m_aa1.nodes_db.node[["v_Intercept", "v_C(mask)[T.lower]", "v_C(mask)[T.upper]"]]
hddm.analyze.plot_posterior_nodes([aa1_v_none, aa1_v_lower, aa1_v_upper])
plt.savefig('aa1_5000/faceMask_hddm_drift_maskFalsePositives_aa1_5000_v_mask.pdf')

In [ ]:
## estimate probabilities that mask coefficient posteriors differ from 0
# note that comparison condition coefficients (i.e. lower/upper) are *relative to* baseline condition (i.e. no mask)
# for lower/upper, =0 means no change from baseline, <0 means less than baseline, and >0 means greater than baseline
# for no mask, =0 means null drift, <0 means negative drift, and >0 means positive drift
print("P(aa1_v_none > 0) = ", (aa1_v_none.trace() > 0).mean())
print("P(aa1_v_lower < 0) = ", (aa1_v_lower.trace() < 0).mean())
print("P(aa1_v_upper > 0) = ", (aa1_v_upper.trace() > 0).mean())
# estimate probability that lower and upper mask coefficient posteriors differ
print("P(aa1_v_lower < aa1_v_upper) = ", (aa1_v_lower.trace() < aa1_v_upper.trace()).mean())

In [ ]:
# load hDDM-processed data from CSV file into a NumPy structured array
data2 = hddm.load_csv('faceMask2_hddm_proc.csv')
                     
# flip 'no' response RTs to be negative
data2 = hddm.utils.flip_errors(data2)

# check dataset
data2.head(10)

In [ ]:
# plot RTs for each participant
fig = plt.figure()
ax = fig.add_subplot(111, xlabel='RT', ylabel='count', title='RT distributions')
for i, subj_data in data2.groupby('subj_idx'):
    subj_data.rt.hist(bins=20, histtype='step', ax=ax)

In [ ]:
# subset dataset by emotion rating type (angry) and facial expression (angry)
aa2_data = data2.loc[(data2['emotionRating'] == 'angry') & (data2['expression'] == 'angry')]
aa2_data.head(10)

In [ ]:
# drop any subjects missing columns in design matrix (otherwise produces error)
aa2_data = aa2_data[(aa2_data.subj_idx != 'bmlp46ta94pzt5x') & (aa2_data.subj_idx != 'dqph4ch0mkofejw') & (aa2_data.subj_idx != 'me8doxrmo9vj9dx')]

In [ ]:
# define model
m_aa2 = hddm.HDDMRegressor(aa2_data, "v ~ C(mask)", bias=True, p_outlier=0.05)

In [ ]:
# run model
m_aa2.sample(5000, burn=200, dbname='traces.db', db='pickle')
m_aa2.print_stats()
m_aa2.plot_posteriors()

In [ ]:
# save model stats
aa2_stats = m_aa2.gen_stats()
print(aa2_stats)
aa2_stats.to_csv('aa2_5000/faceMask2_hddm_drift_maskFalseNegatives_aa2_5000.csv', index=True)

# save model posterior plots
m_aa2.plot_posteriors(path='aa2_5000/_posteriors', save=True)

In [ ]:
# plot model posteriors by mask
aa2_v_none, aa2_v_lower, aa2_v_upper = m_aa2.nodes_db.node[["v_Intercept", "v_C(mask)[T.lower]", "v_C(mask)[T.upper]"]]
hddm.analyze.plot_posterior_nodes([aa2_v_none, aa2_v_lower, aa2_v_upper])
plt.savefig('aa2_5000/faceMask2_hddm_drift_maskFalsePositives_aa2_5000_v_mask.pdf')

In [ ]:
## estimate probabilities that mask coefficient posteriors differ from 0
# note that comparison condition coefficients (i.e. lower/upper) are *relative to* baseline condition (i.e. no mask)
# for lower/upper, =0 means no change from baseline, <0 means less than baseline, and >0 means greater than baseline
# for no mask, =0 means null drift, <0 means negative drift, and >0 means positive drift
print("P(aa2_v_none < 0) = ", (aa2_v_none.trace() < 0).mean())
print("P(aa2_v_lower > 0) = ", (aa2_v_lower.trace() > 0).mean())
print("P(aa2_v_upper > 0) = ", (aa2_v_upper.trace() > 0).mean())
# estimate probability that lower and upper mask coefficient posteriors differ
print("P(aa2_v_lower < aa2_v_upper) = ", (aa2_v_lower.trace() < aa2_v_upper.trace()).mean())

In [ ]:
## estimate probabilities that mask coefficient posteriors differ from each other (faceMask 1 vs. faceMask2)
# note that comparison condition coefficients (i.e. lower/upper) are *relative to* baseline condition (i.e. no mask)
# for lower/upper, =0 means no change from baseline, <0 means less than baseline, and >0 means greater than baseline
# for no mask, =0 means null drift, <0 means negative drift, and >0 means positive drift
print("P(aa1_v_none > aa2_v_none) = ", ((aa1_v_none.trace() > aa2_v_none.trace()).mean()))
print("P(aa1_v_lower < aa2_v_lower) = ", ((aa1_v_lower.trace() < aa2_v_lower.trace()).mean()))
print("P(aa1_v_upper < aa2_v_upper) = ", ((aa1_v_upper.trace() < aa2_v_upper.trace()).mean()))


In [ ]:
## adding together the relative values:
aa1_v_none_plus_lower = aa1_v_none.trace() + aa1_v_lower.trace()
aa1_v_none_plus_upper = aa1_v_none.trace() + aa1_v_upper.trace()
aa2_v_none_plus_lower = aa2_v_none.trace() + aa2_v_lower.trace()
aa2_v_none_plus_upper = aa2_v_none.trace() + aa2_v_upper.trace()

In [ ]:
print("P(aa1_v_none_plus_lower > aa2_v_none_plus_lower) = ", (aa1_v_none_plus_lower > aa2_v_none_plus_lower).mean())
print("P(aa1_v_none_plus_upper > aa2_v_none_plus_upper) = ", (aa1_v_none_plus_upper > aa2_v_none_plus_upper).mean())